In [1]:
import os
import pandas as pd


In [2]:
directory=r'C:\Users\p_uli\Desktop\Columbia University\Cursos\Fall 22\Capstone\Data\maildir'

subset=['taylor-m','fischer-m','buy-r','ybarbo-p','geaccone-t','allen-p']

msg_id= list()
msg_txt= list()

for folder in subset:
    for roots, dirs, files in os.walk(directory+"\\"+folder):
        for file in files:
            with open(roots+'\\'+file) as f:
                x= f.readlines()
                x=''.join(x)
                msg_txt.append(x)
                msg_id.append(roots[roots.index(folder):] + '\\'+file)


In [3]:
data=pd.DataFrame(zip(msg_id,msg_txt),columns=['message_id','message_text'])
data.head()

,message_id,message_text
0,taylor-m\all_documents\1,Message-ID: <25895411.1075859874878.JavaMail.e...
1,taylor-m\all_documents\10,Message-ID: <23416266.1075859875079.JavaMail.e...
2,taylor-m\all_documents\100,Message-ID: <20401126.1075859877231.JavaMail.e...
3,taylor-m\all_documents\1000,Message-ID: <9557191.1075859900977.JavaMail.ev...
4,taylor-m\all_documents\1001,Message-ID: <2650318.1075859901001.JavaMail.ev...


In [6]:
#data.to_csv(r'C:\Users\p_uli\Desktop\Columbia University\Cursos\Fall 22\Capstone\Data\subset.csv',index=False)
import pickle


(23719, 2)